In [ ]:
#scrap post data from user account: muyangquandebaba
import requests
import pandas as pd
from datetime import datetime, timezone
from sqlalchemy import create_engine, text  
import pytz

#Create a connection to database   
engine = create_engine('mysql+pymysql://masterUsername:xxxxxxxx')

#Specify the start datetime from which you want to scrape your data
createtime_normal = '20240501000000' 
createtime_normal_formatted = datetime.strptime(createtime_normal, '%Y%m%d%H%M%S') 

#Convert the datetime object to a Unix timestamp for using EnsembleData API
createtime_utc_timezone_in_unix = int(createtime_normal_formatted.timestamp())

#Get user post data from EnsembleData API 
root = "https://www.ensembledata.com/apis"
endpoint = "/tt/user/posts-from-secuid"
params = {
  "secUid": "MS4wLjABAAAAr7DNgQlzyaZkRiduvBJNS6qNyGrNiVN-imkNEGdEeNE1E6CKNPp3e2kXIfEQ9rKE",
  "depth": 600,  #not necessarily be one post, it depends on the endpoint 
  "start_cursor": 0,
  "oldest_createtime":createtime_utc_timezone_in_unix,
  "token": "xxxxxxx"
}

res = requests.get(root+endpoint, params=params)
muyangquandebaba = res.json()

#flatten a nested JSON structure
data_frame = pd.json_normalize(muyangquandebaba['data'])

# Convert Unix time to 'yyyymmdd' format  
data_frame['create_time'] = pd.to_datetime(data_frame['create_time'], unit='s')

# Convert to another time zone
data_frame['create_time'] = data_frame['create_time'].dt.tz_localize('UTC').dt.tz_convert('Asia/Shanghai')
data_frame['create_time'] = data_frame['create_time'].dt.strftime('%Y%m%d%H%M%S')

# Select specific columns
selected_columns = data_frame[['aweme_id','create_time','content_desc','follow_up_item_id_groups','follow_up_publish_from_id','geofencing_regions','user_digged']]
selected_columns.to_sql('muyangquandebaba_post_table',engine,chunksize=100000,index=None) #store all comment    

In [ ]:
#Scrap comment data from the TikTok page: muyangquandebaba
import requests
import pandas as pd
from datetime import datetime
from sqlalchemy import create_engine, text

#Create a connection to the MySQL database using SQLAlchemy
engine = create_engine('mysql+pymysql://masterUsername:xxxxxxxx')

# Query the MySQL table and load the result into a Pandas DataFrame
table_name = 'muyangquandebaba_post_table'
query = f"SELECT aweme_id FROM {table_name}"
data_frame_from_mysql = pd.read_sql(query, con=engine)

# Loop through each aweme_id in the DataFrame
for aweme_id in data_frame_from_mysql['aweme_id']:
    
    root = "https://www.ensembledata.com/apis"
    endpoint = "/tt/post/comments"
    params = { 
       "aweme_id": aweme_id,
       "cursor": 0,
       "token": "xxxxxxxxxx" 
       }
    
    res = requests.get(root+endpoint, params=params) 
    print(f'now process aweme_id: {aweme_id}')
    
    if res.status_code == 200:
        muyangquandebaba_post_comments = res.json()  
        
        if 'data' in muyangquandebaba_post_comments and muyangquandebaba_post_comments['data'] is not None:
    # Check if 'comments' key exists in lkcyedan_post_table['data']
            if 'comments' in muyangquandebaba_post_comments['data']:
                data_frame = pd.json_normalize(muyangquandebaba_post_comments['data']['comments'])
                # Continue with your processing
                data_frame['create_time'] = pd.to_datetime(data_frame['create_time'], unit='s').dt.strftime('%Y%m%d%H%M%S')
                data_frame.astype(str).to_sql('muyangquandebaba_post_comments_temp', engine, chunksize=100000, index=None)
            else:
                print(f"No 'comments' key in muyangquandebaba_post_comments['data']")
                continue
        else:
            print("No 'data' key in muyangquandebaba_post_comments")
            continue

        
    
        sql_insert_into_another_table = text("""
INSERT IGNORE INTO muyangquandebaba_post_comments (
`author_pin`,
`aweme_id`,
`cid`,
`collect_stat`,
`comment_language`,
`comment_post_item_ids`,
`create_time`,
`digg_count`,
`image_list`,
`is_author_digged`,
`label_list`,
`no_show`,
`reply_comment`,
`reply_comment_total`,
`reply_id`,
`reply_to_reply_id`,
`share_info.acl.code`,
`share_info.acl.extra`,
`share_info.desc`,
`share_info.title`,
`share_info.url`,
`status`,
`stick_position`,
`text`,
`text_extra`,
`trans_btn_style`,
`user.accept_private_policy`,
`user.account_labels`,
`user.account_region`,
`user.ad_cover_url`,
`user.advance_feature_item_order`,
`user.advanced_feature_info`,
`user.authority_status`,
`user.avatar_thumb.height`,
`user.avatar_thumb.uri`,
`user.avatar_thumb.url_list`,
`user.avatar_thumb.url_prefix`,
`user.avatar_thumb.width`,
`user.avatar_uri`,
`user.aweme_count`,
`user.bind_phone`,
`user.birthday`,
`user.bold_fields`,
`user.can_message_follow_status_list`,
`user.can_set_geofencing`,
`user.cha_list`,
`user.comment_filter_status`,
`user.comment_setting`,
`user.commerce_user_level`,
`user.cover_url`,
`user.create_time`,
`user.custom_verify`,
`user.cv_level`,
`user.download_prompt_ts`,
`user.download_setting`,
`user.duet_setting`,
`user.enterprise_verify_reason`,
`user.events`,
`user.favoriting_count`,
`user.follow_status`,
`user.follower_count`,
`user.follower_status`,
`user.followers_detail`,
`user.following_count`,
`user.friends_status`,
`user.gender`,
`user.geofencing`,
`user.has_email`,
`user.has_insights`,
`user.has_orders`,
`user.hide_search`,
`user.homepage_bottom_toast`,
`user.is_ad_fake`,
`user.is_block`,
`user.is_discipline_member`,
`user.is_phone_binded`,
`user.is_star`,
`user.item_list`,
`user.language`,
`user.live_agreement`,
`user.live_agreement_time`,
`user.live_commerce`,
`user.live_verify`,
`user.matched_friend_available`,
`user.mention_status`,
`user.mutual_relation_avatars`,
`user.need_points`,
`user.need_recommend`,
`user.nickname`,
`user.platform_sync_info`,
`user.prevent_download`,
`user.react_setting`,
`user.reflow_page_gid`,
`user.reflow_page_uid`,
`user.region`,
`user.relative_users`,
`user.search_highlight`,
`user.sec_uid`,
`user.secret`,
`user.shield_comment_notice`,
`user.shield_digg_notice`,
`user.shield_edit_field_info`,
`user.shield_follow_notice`,
`user.short_id`,
`user.show_image_bubble`,
`user.signature`,
`user.special_account.special_account_list`,
`user.special_lock`,
`user.status`,
`user.stitch_setting`,
`user.total_favorited`,
`user.twitter_id`,
`user.twitter_name`,
`user.type_label`,
`user.uid`,
`user.unique_id`,
`user.unique_id_modify_time`,
`user.user_canceled`,
`user.user_mode`,
`user.user_period`,
`user.user_profile_guide`,
`user.user_rate`,
`user.user_tags`,
`user.verification_type`,
`user.verify_info`,
`user.video_icon.height`,
`user.video_icon.uri`,
`user.video_icon.url_list`,
`user.video_icon.url_prefix`,
`user.video_icon.width`,
`user.white_cover_url`,
`user.with_commerce_entry`,
`user.with_fusion_shop_entry`,
`user.with_shop_entry`,
`user.youtube_channel_id`,
`user.youtube_channel_title`,
`user.youtube_expire_time`,
`user_buried`,
`user_digged`
)
select 
`author_pin`,
`aweme_id`,
`cid`,
`collect_stat`,
`comment_language`,
`comment_post_item_ids`,
`create_time`,
`digg_count`,
`image_list`,
`is_author_digged`,
`label_list`,
`no_show`,
`reply_comment`,
`reply_comment_total`,
`reply_id`,
`reply_to_reply_id`,
`share_info.acl.code`,
`share_info.acl.extra`,
`share_info.desc`,
`share_info.title`,
`share_info.url`,
`status`,
`stick_position`,
`text`,
`text_extra`,
`trans_btn_style`,
`user.accept_private_policy`,
`user.account_labels`,
`user.account_region`,
`user.ad_cover_url`,
`user.advance_feature_item_order`,
`user.advanced_feature_info`,
`user.authority_status`,
`user.avatar_thumb.height`,
`user.avatar_thumb.uri`,
`user.avatar_thumb.url_list`,
`user.avatar_thumb.url_prefix`,
`user.avatar_thumb.width`,
`user.avatar_uri`,
`user.aweme_count`,
`user.bind_phone`,
`user.birthday`,
`user.bold_fields`,
`user.can_message_follow_status_list`,
`user.can_set_geofencing`,
`user.cha_list`,
`user.comment_filter_status`,
`user.comment_setting`,
`user.commerce_user_level`,
`user.cover_url`,
`user.create_time`,
`user.custom_verify`,
`user.cv_level`,
`user.download_prompt_ts`,
`user.download_setting`,
`user.duet_setting`,
`user.enterprise_verify_reason`,
`user.events`,
`user.favoriting_count`,
`user.follow_status`,
`user.follower_count`,
`user.follower_status`,
`user.followers_detail`,
`user.following_count`,
`user.friends_status`,
`user.gender`,
`user.geofencing`,
`user.has_email`,
`user.has_insights`,
`user.has_orders`,
`user.hide_search`,
`user.homepage_bottom_toast`,
`user.is_ad_fake`,
`user.is_block`,
`user.is_discipline_member`,
`user.is_phone_binded`,
`user.is_star`,
`user.item_list`,
`user.language`,
`user.live_agreement`,
`user.live_agreement_time`,
`user.live_commerce`,
`user.live_verify`,
`user.matched_friend_available`,
`user.mention_status`,
`user.mutual_relation_avatars`,
`user.need_points`,
`user.need_recommend`,
`user.nickname`,
`user.platform_sync_info`,
`user.prevent_download`,
`user.react_setting`,
`user.reflow_page_gid`,
`user.reflow_page_uid`,
`user.region`,
`user.relative_users`,
`user.search_highlight`,
`user.sec_uid`,
`user.secret`,
`user.shield_comment_notice`,
`user.shield_digg_notice`,
`user.shield_edit_field_info`,
`user.shield_follow_notice`,
`user.short_id`,
`user.show_image_bubble`,
`user.signature`,
`user.special_account.special_account_list`,
`user.special_lock`,
`user.status`,
`user.stitch_setting`,
`user.total_favorited`,
`user.twitter_id`,
`user.twitter_name`,
`user.type_label`,
`user.uid`,
`user.unique_id`,
`user.unique_id_modify_time`,
`user.user_canceled`,
`user.user_mode`,
`user.user_period`,
`user.user_profile_guide`,
`user.user_rate`,
`user.user_tags`,
`user.verification_type`,
`user.verify_info`,
`user.video_icon.height`,
`user.video_icon.uri`,
`user.video_icon.url_list`,
`user.video_icon.url_prefix`,
`user.video_icon.width`,
`user.white_cover_url`,
`user.with_commerce_entry`,
`user.with_fusion_shop_entry`,
`user.with_shop_entry`,
`user.youtube_channel_id`,
`user.youtube_channel_title`,
`user.youtube_expire_time`,
`user_buried`,
`user_digged`

from true_nora1111_post_comments_temp temp 

ON DUPLICATE KEY UPDATE
INSERT IGNORE INTO muyangquandebaba_post_comments (
`author_pin`,
`aweme_id`,
`cid`,
`collect_stat`,
`comment_language`,
`comment_post_item_ids`,
`create_time`,
`digg_count`,
`image_list`,
`is_author_digged`,
`label_list`,
`no_show`,
`reply_comment`,
`reply_comment_total`,
`reply_id`,
`reply_to_reply_id`,
`share_info.acl.code`,
`share_info.acl.extra`,
`share_info.desc`,
`share_info.title`,
`share_info.url`,
`status`,
`stick_position`,
`text`,
`text_extra`,
`trans_btn_style`,
`user.accept_private_policy`,
`user.account_labels`,
`user.account_region`,
`user.ad_cover_url`,
`user.advance_feature_item_order`,
`user.advanced_feature_info`,
`user.authority_status`,
`user.avatar_thumb.height`,
`user.avatar_thumb.uri`,
`user.avatar_thumb.url_list`,
`user.avatar_thumb.url_prefix`,
`user.avatar_thumb.width`,
`user.avatar_uri`,
`user.aweme_count`,
`user.bind_phone`,
`user.birthday`,
`user.bold_fields`,
`user.can_message_follow_status_list`,
`user.can_set_geofencing`,
`user.cha_list`,
`user.comment_filter_status`,
`user.comment_setting`,
`user.commerce_user_level`,
`user.cover_url`,
`user.create_time`,
`user.custom_verify`,
`user.cv_level`,
`user.download_prompt_ts`,
`user.download_setting`,
`user.duet_setting`,
`user.enterprise_verify_reason`,
`user.events`,
`user.favoriting_count`,
`user.follow_status`,
`user.follower_count`,
`user.follower_status`,
`user.followers_detail`,
`user.following_count`,
`user.friends_status`,
`user.gender`,
`user.geofencing`,
`user.has_email`,
`user.has_insights`,
`user.has_orders`,
`user.hide_search`,
`user.homepage_bottom_toast`,
`user.is_ad_fake`,
`user.is_block`,
`user.is_discipline_member`,
`user.is_phone_binded`,
`user.is_star`,
`user.item_list`,
`user.language`,
`user.live_agreement`,
`user.live_agreement_time`,
`user.live_commerce`,
`user.live_verify`,
`user.matched_friend_available`,
`user.mention_status`,
`user.mutual_relation_avatars`,
`user.need_points`,
`user.need_recommend`,
`user.nickname`,
`user.platform_sync_info`,
`user.prevent_download`,
`user.react_setting`,
`user.reflow_page_gid`,
`user.reflow_page_uid`,
`user.region`,
`user.relative_users`,
`user.search_highlight`,
`user.sec_uid`,
`user.secret`,
`user.shield_comment_notice`,
`user.shield_digg_notice`,
`user.shield_edit_field_info`,
`user.shield_follow_notice`,
`user.short_id`,
`user.show_image_bubble`,
`user.signature`,
`user.special_account.special_account_list`,
`user.special_lock`,
`user.status`,
`user.stitch_setting`,
`user.total_favorited`,
`user.twitter_id`,
`user.twitter_name`,
`user.type_label`,
`user.uid`,
`user.unique_id`,
`user.unique_id_modify_time`,
`user.user_canceled`,
`user.user_mode`,
`user.user_period`,
`user.user_profile_guide`,
`user.user_rate`,
`user.user_tags`,
`user.verification_type`,
`user.verify_info`,
`user.video_icon.height`,
`user.video_icon.uri`,
`user.video_icon.url_list`,
`user.video_icon.url_prefix`,
`user.video_icon.width`,
`user.white_cover_url`,
`user.with_commerce_entry`,
`user.with_fusion_shop_entry`,
`user.with_shop_entry`,
`user.youtube_channel_id`,
`user.youtube_channel_title`,
`user.youtube_expire_time`,
`user_buried`,
`user_digged`
)
select 
`author_pin`,
`aweme_id`,
`cid`,
`collect_stat`,
`comment_language`,
`comment_post_item_ids`,
`create_time`,
`digg_count`,
`image_list`,
`is_author_digged`,
`label_list`,
`no_show`,
`reply_comment`,
`reply_comment_total`,
`reply_id`,
`reply_to_reply_id`,
`share_info.acl.code`,
`share_info.acl.extra`,
`share_info.desc`,
`share_info.title`,
`share_info.url`,
`status`,
`stick_position`,
`text`,
`text_extra`,
`trans_btn_style`,
`user.accept_private_policy`,
`user.account_labels`,
`user.account_region`,
`user.ad_cover_url`,
`user.advance_feature_item_order`,
`user.advanced_feature_info`,
`user.authority_status`,
`user.avatar_thumb.height`,
`user.avatar_thumb.uri`,
`user.avatar_thumb.url_list`,
`user.avatar_thumb.url_prefix`,
`user.avatar_thumb.width`,
`user.avatar_uri`,
`user.aweme_count`,
`user.bind_phone`,
`user.birthday`,
`user.bold_fields`,
`user.can_message_follow_status_list`,
`user.can_set_geofencing`,
`user.cha_list`,
`user.comment_filter_status`,
`user.comment_setting`,
`user.commerce_user_level`,
`user.cover_url`,
`user.create_time`,
`user.custom_verify`,
`user.cv_level`,
`user.download_prompt_ts`,
`user.download_setting`,
`user.duet_setting`,
`user.enterprise_verify_reason`,
`user.events`,
`user.favoriting_count`,
`user.follow_status`,
`user.follower_count`,
`user.follower_status`,
`user.followers_detail`,
`user.following_count`,
`user.friends_status`,
`user.gender`,
`user.geofencing`,
`user.has_email`,
`user.has_insights`,
`user.has_orders`,
`user.hide_search`,
`user.homepage_bottom_toast`,
`user.is_ad_fake`,
`user.is_block`,
`user.is_discipline_member`,
`user.is_phone_binded`,
`user.is_star`,
`user.item_list`,
`user.language`,
`user.live_agreement`,
`user.live_agreement_time`,
`user.live_commerce`,
`user.live_verify`,
`user.matched_friend_available`,
`user.mention_status`,
`user.mutual_relation_avatars`,
`user.need_points`,
`user.need_recommend`,
`user.nickname`,
`user.platform_sync_info`,
`user.prevent_download`,
`user.react_setting`,
`user.reflow_page_gid`,
`user.reflow_page_uid`,
`user.region`,
`user.relative_users`,
`user.search_highlight`,
`user.sec_uid`,
`user.secret`,
`user.shield_comment_notice`,
`user.shield_digg_notice`,
`user.shield_edit_field_info`,
`user.shield_follow_notice`,
`user.short_id`,
`user.show_image_bubble`,
`user.signature`,
`user.special_account.special_account_list`,
`user.special_lock`,
`user.status`,
`user.stitch_setting`,
`user.total_favorited`,
`user.twitter_id`,
`user.twitter_name`,
`user.type_label`,
`user.uid`,
`user.unique_id`,
`user.unique_id_modify_time`,
`user.user_canceled`,
`user.user_mode`,
`user.user_period`,
`user.user_profile_guide`,
`user.user_rate`,
`user.user_tags`,
`user.verification_type`,
`user.verify_info`,
`user.video_icon.height`,
`user.video_icon.uri`,
`user.video_icon.url_list`,
`user.video_icon.url_prefix`,
`user.video_icon.width`,
`user.white_cover_url`,
`user.with_commerce_entry`,
`user.with_fusion_shop_entry`,
`user.with_shop_entry`,
`user.youtube_channel_id`,
`user.youtube_channel_title`,
`user.youtube_expire_time`,
`user_buried`,
`user_digged`

from muyangquandebaba_post_comments_temp temp 

ON DUPLICATE KEY UPDATE
`muyangquandebaba_post_comments`.`author_pin` = COALESCE(VALUES(`author_pin`),`muyangquandebaba_post_comments`.`author_pin`),
`muyangquandebaba_post_comments`.`aweme_id` = COALESCE(VALUES(`aweme_id`),`muyangquandebaba_post_comments`.`aweme_id`),
`muyangquandebaba_post_comments`.`cid` = COALESCE(VALUES(`cid`),`muyangquandebaba_post_comments`.`cid`),
`muyangquandebaba_post_comments`.`collect_stat` = COALESCE(VALUES(`collect_stat`),`muyangquandebaba_post_comments`.`collect_stat`),
`muyangquandebaba_post_comments`.`comment_language` = COALESCE(VALUES(`comment_language`),`muyangquandebaba_post_comments`.`comment_language`),
`muyangquandebaba_post_comments`.`comment_post_item_ids` = COALESCE(VALUES(`comment_post_item_ids`),`muyangquandebaba_post_comments`.`comment_post_item_ids`),
`muyangquandebaba_post_comments`.`create_time` = COALESCE(VALUES(`create_time`),`muyangquandebaba_post_comments`.`create_time`),
`muyangquandebaba_post_comments`.`digg_count` = COALESCE(VALUES(`digg_count`),`muyangquandebaba_post_comments`.`digg_count`),
`muyangquandebaba_post_comments`.`image_list` = COALESCE(VALUES(`image_list`),`muyangquandebaba_post_comments`.`image_list`),
`muyangquandebaba_post_comments`.`is_author_digged` = COALESCE(VALUES(`is_author_digged`),`muyangquandebaba_post_comments`.`is_author_digged`),
`muyangquandebaba_post_comments`.`label_list` = COALESCE(VALUES(`label_list`),`muyangquandebaba_post_comments`.`label_list`),
`muyangquandebaba_post_comments`.`no_show` = COALESCE(VALUES(`no_show`),`muyangquandebaba_post_comments`.`no_show`),
`muyangquandebaba_post_comments`.`reply_comment` = COALESCE(VALUES(`reply_comment`),`muyangquandebaba_post_comments`.`reply_comment`),
`muyangquandebaba_post_comments`.`reply_comment_total` = COALESCE(VALUES(`reply_comment_total`),`muyangquandebaba_post_comments`.`reply_comment_total`),
`muyangquandebaba_post_comments`.`reply_id` = COALESCE(VALUES(`reply_id`),`muyangquandebaba_post_comments`.`reply_id`),
`muyangquandebaba_post_comments`.`reply_to_reply_id` = COALESCE(VALUES(`reply_to_reply_id`),`muyangquandebaba_post_comments`.`reply_to_reply_id`),
`muyangquandebaba_post_comments`.`share_info.acl.code` = COALESCE(VALUES(`share_info.acl.code`),`muyangquandebaba_post_comments`.`share_info.acl.code`),
`muyangquandebaba_post_comments`.`share_info.acl.extra` = COALESCE(VALUES(`share_info.acl.extra`),`muyangquandebaba_post_comments`.`share_info.acl.extra`),
`muyangquandebaba_post_comments`.`share_info.desc` = COALESCE(VALUES(`share_info.desc`),`muyangquandebaba_post_comments`.`share_info.desc`),
`muyangquandebaba_post_comments`.`share_info.title` = COALESCE(VALUES(`share_info.title`),`muyangquandebaba_post_comments`.`share_info.title`),
`muyangquandebaba_post_comments`.`share_info.url` = COALESCE(VALUES(`share_info.url`),`muyangquandebaba_post_comments`.`share_info.url`),
`muyangquandebaba_post_comments`.`status` = COALESCE(VALUES(`status`),`muyangquandebaba_post_comments`.`status`),
`muyangquandebaba_post_comments`.`stick_position` = COALESCE(VALUES(`stick_position`),`muyangquandebaba_post_comments`.`stick_position`),
`muyangquandebaba_post_comments`.`text` = COALESCE(VALUES(`text`),`muyangquandebaba_post_comments`.`text`),
`muyangquandebaba_post_comments`.`text_extra` = COALESCE(VALUES(`text_extra`),`muyangquandebaba_post_comments`.`text_extra`),
`muyangquandebaba_post_comments`.`trans_btn_style` = COALESCE(VALUES(`trans_btn_style`),`muyangquandebaba_post_comments`.`trans_btn_style`),
`muyangquandebaba_post_comments`.`user.accept_private_policy` = COALESCE(VALUES(`user.accept_private_policy`),`muyangquandebaba_post_comments`.`user.accept_private_policy`),
`muyangquandebaba_post_comments`.`user.account_labels` = COALESCE(VALUES(`user.account_labels`),`muyangquandebaba_post_comments`.`user.account_labels`),
`muyangquandebaba_post_comments`.`user.account_region` = COALESCE(VALUES(`user.account_region`),`muyangquandebaba_post_comments`.`user.account_region`),
`muyangquandebaba_post_comments`.`user.ad_cover_url` = COALESCE(VALUES(`user.ad_cover_url`),`muyangquandebaba_post_comments`.`user.ad_cover_url`),
`muyangquandebaba_post_comments`.`user.advance_feature_item_order` = COALESCE(VALUES(`user.advance_feature_item_order`),`muyangquandebaba_post_comments`.`user.advance_feature_item_order`),
`muyangquandebaba_post_comments`.`user.advanced_feature_info` = COALESCE(VALUES(`user.advanced_feature_info`),`muyangquandebaba_post_comments`.`user.advanced_feature_info`),
`muyangquandebaba_post_comments`.`user.authority_status` = COALESCE(VALUES(`user.authority_status`),`muyangquandebaba_post_comments`.`user.authority_status`),
`muyangquandebaba_post_comments`.`user.avatar_thumb.height` = COALESCE(VALUES(`user.avatar_thumb.height`),`muyangquandebaba_post_comments`.`user.avatar_thumb.height`),
`muyangquandebaba_post_comments`.`user.avatar_thumb.uri` = COALESCE(VALUES(`user.avatar_thumb.uri`),`muyangquandebaba_post_comments`.`user.avatar_thumb.uri`),
`muyangquandebaba_post_comments`.`user.avatar_thumb.url_list` = COALESCE(VALUES(`user.avatar_thumb.url_list`),`muyangquandebaba_post_comments`.`user.avatar_thumb.url_list`),
`muyangquandebaba_post_comments`.`user.avatar_thumb.url_prefix` = COALESCE(VALUES(`user.avatar_thumb.url_prefix`),`muyangquandebaba_post_comments`.`user.avatar_thumb.url_prefix`),
`muyangquandebaba_post_comments`.`user.avatar_thumb.width` = COALESCE(VALUES(`user.avatar_thumb.width`),`muyangquandebaba_post_comments`.`user.avatar_thumb.width`),
`muyangquandebaba_post_comments`.`user.avatar_uri` = COALESCE(VALUES(`user.avatar_uri`),`muyangquandebaba_post_comments`.`user.avatar_uri`),
`muyangquandebaba_post_comments`.`user.aweme_count` = COALESCE(VALUES(`user.aweme_count`),`muyangquandebaba_post_comments`.`user.aweme_count`),
`muyangquandebaba_post_comments`.`user.bind_phone` = COALESCE(VALUES(`user.bind_phone`),`muyangquandebaba_post_comments`.`user.bind_phone`),
`muyangquandebaba_post_comments`.`user.birthday` = COALESCE(VALUES(`user.birthday`),`muyangquandebaba_post_comments`.`user.birthday`),
`muyangquandebaba_post_comments`.`user.bold_fields` = COALESCE(VALUES(`user.bold_fields`),`muyangquandebaba_post_comments`.`user.bold_fields`),
`muyangquandebaba_post_comments`.`user.can_message_follow_status_list` = COALESCE(VALUES(`user.can_message_follow_status_list`),`muyangquandebaba_post_comments`.`user.can_message_follow_status_list`),
`muyangquandebaba_post_comments`.`user.can_set_geofencing` = COALESCE(VALUES(`user.can_set_geofencing`),`muyangquandebaba_post_comments`.`user.can_set_geofencing`),
`muyangquandebaba_post_comments`.`user.cha_list` = COALESCE(VALUES(`user.cha_list`),`muyangquandebaba_post_comments`.`user.cha_list`),
`muyangquandebaba_post_comments`.`user.comment_filter_status` = COALESCE(VALUES(`user.comment_filter_status`),`muyangquandebaba_post_comments`.`user.comment_filter_status`),
`muyangquandebaba_post_comments`.`user.comment_setting` = COALESCE(VALUES(`user.comment_setting`),`muyangquandebaba_post_comments`.`user.comment_setting`),
`muyangquandebaba_post_comments`.`user.commerce_user_level` = COALESCE(VALUES(`user.commerce_user_level`),`muyangquandebaba_post_comments`.`user.commerce_user_level`),
`muyangquandebaba_post_comments`.`user.cover_url` = COALESCE(VALUES(`user.cover_url`),`muyangquandebaba_post_comments`.`user.cover_url`),
`muyangquandebaba_post_comments`.`user.create_time` = COALESCE(VALUES(`user.create_time`),`muyangquandebaba_post_comments`.`user.create_time`),
`muyangquandebaba_post_comments`.`user.custom_verify` = COALESCE(VALUES(`user.custom_verify`),`muyangquandebaba_post_comments`.`user.custom_verify`),
`muyangquandebaba_post_comments`.`user.cv_level` = COALESCE(VALUES(`user.cv_level`),`muyangquandebaba_post_comments`.`user.cv_level`),
`muyangquandebaba_post_comments`.`user.download_prompt_ts` = COALESCE(VALUES(`user.download_prompt_ts`),`muyangquandebaba_post_comments`.`user.download_prompt_ts`),
`muyangquandebaba_post_comments`.`user.download_setting` = COALESCE(VALUES(`user.download_setting`),`muyangquandebaba_post_comments`.`user.download_setting`),
`muyangquandebaba_post_comments`.`user.duet_setting` = COALESCE(VALUES(`user.duet_setting`),`muyangquandebaba_post_comments`.`user.duet_setting`),
`muyangquandebaba_post_comments`.`user.enterprise_verify_reason` = COALESCE(VALUES(`user.enterprise_verify_reason`),`muyangquandebaba_post_comments`.`user.enterprise_verify_reason`),
`muyangquandebaba_post_comments`.`user.events` = COALESCE(VALUES(`user.events`),`muyangquandebaba_post_comments`.`user.events`),
`muyangquandebaba_post_comments`.`user.favoriting_count` = COALESCE(VALUES(`user.favoriting_count`),`muyangquandebaba_post_comments`.`user.favoriting_count`),
`muyangquandebaba_post_comments`.`user.follow_status` = COALESCE(VALUES(`user.follow_status`),`muyangquandebaba_post_comments`.`user.follow_status`),
`muyangquandebaba_post_comments`.`user.follower_count` = COALESCE(VALUES(`user.follower_count`),`muyangquandebaba_post_comments`.`user.follower_count`),
`muyangquandebaba_post_comments`.`user.follower_status` = COALESCE(VALUES(`user.follower_status`),`muyangquandebaba_post_comments`.`user.follower_status`),
`muyangquandebaba_post_comments`.`user.followers_detail` = COALESCE(VALUES(`user.followers_detail`),`muyangquandebaba_post_comments`.`user.followers_detail`),
`muyangquandebaba_post_comments`.`user.following_count` = COALESCE(VALUES(`user.following_count`),`muyangquandebaba_post_comments`.`user.following_count`),
`muyangquandebaba_post_comments`.`user.friends_status` = COALESCE(VALUES(`user.friends_status`),`muyangquandebaba_post_comments`.`user.friends_status`),
`muyangquandebaba_post_comments`.`user.gender` = COALESCE(VALUES(`user.gender`),`muyangquandebaba_post_comments`.`user.gender`),
`muyangquandebaba_post_comments`.`user.geofencing` = COALESCE(VALUES(`user.geofencing`),`muyangquandebaba_post_comments`.`user.geofencing`),
`muyangquandebaba_post_comments`.`user.has_email` = COALESCE(VALUES(`user.has_email`),`muyangquandebaba_post_comments`.`user.has_email`),
`muyangquandebaba_post_comments`.`user.has_insights` = COALESCE(VALUES(`user.has_insights`),`muyangquandebaba_post_comments`.`user.has_insights`),
`muyangquandebaba_post_comments`.`user.has_orders` = COALESCE(VALUES(`user.has_orders`),`muyangquandebaba_post_comments`.`user.has_orders`),
`muyangquandebaba_post_comments`.`user.hide_search` = COALESCE(VALUES(`user.hide_search`),`muyangquandebaba_post_comments`.`user.hide_search`),
`muyangquandebaba_post_comments`.`user.homepage_bottom_toast` = COALESCE(VALUES(`user.homepage_bottom_toast`),`muyangquandebaba_post_comments`.`user.homepage_bottom_toast`),
`muyangquandebaba_post_comments`.`user.is_ad_fake` = COALESCE(VALUES(`user.is_ad_fake`),`muyangquandebaba_post_comments`.`user.is_ad_fake`),
`muyangquandebaba_post_comments`.`user.is_block` = COALESCE(VALUES(`user.is_block`),`muyangquandebaba_post_comments`.`user.is_block`),
`muyangquandebaba_post_comments`.`user.is_discipline_member` = COALESCE(VALUES(`user.is_discipline_member`),`muyangquandebaba_post_comments`.`user.is_discipline_member`),
`muyangquandebaba_post_comments`.`user.is_phone_binded` = COALESCE(VALUES(`user.is_phone_binded`),`muyangquandebaba_post_comments`.`user.is_phone_binded`),
`muyangquandebaba_post_comments`.`user.is_star` = COALESCE(VALUES(`user.is_star`),`muyangquandebaba_post_comments`.`user.is_star`),
`muyangquandebaba_post_comments`.`user.item_list` = COALESCE(VALUES(`user.item_list`),`muyangquandebaba_post_comments`.`user.item_list`),
`muyangquandebaba_post_comments`.`user.language` = COALESCE(VALUES(`user.language`),`muyangquandebaba_post_comments`.`user.language`),
`muyangquandebaba_post_comments`.`user.live_agreement` = COALESCE(VALUES(`user.live_agreement`),`muyangquandebaba_post_comments`.`user.live_agreement`),
`muyangquandebaba_post_comments`.`user.live_agreement_time` = COALESCE(VALUES(`user.live_agreement_time`),`muyangquandebaba_post_comments`.`user.live_agreement_time`),
`muyangquandebaba_post_comments`.`user.live_commerce` = COALESCE(VALUES(`user.live_commerce`),`muyangquandebaba_post_comments`.`user.live_commerce`),
`muyangquandebaba_post_comments`.`user.live_verify` = COALESCE(VALUES(`user.live_verify`),`muyangquandebaba_post_comments`.`user.live_verify`),
`muyangquandebaba_post_comments`.`user.matched_friend_available` = COALESCE(VALUES(`user.matched_friend_available`),`muyangquandebaba_post_comments`.`user.matched_friend_available`),
`muyangquandebaba_post_comments`.`user.mention_status` = COALESCE(VALUES(`user.mention_status`),`muyangquandebaba_post_comments`.`user.mention_status`),
`muyangquandebaba_post_comments`.`user.mutual_relation_avatars` = COALESCE(VALUES(`user.mutual_relation_avatars`),`muyangquandebaba_post_comments`.`user.mutual_relation_avatars`),
`muyangquandebaba_post_comments`.`user.need_points` = COALESCE(VALUES(`user.need_points`),`muyangquandebaba_post_comments`.`user.need_points`),
`muyangquandebaba_post_comments`.`user.need_recommend` = COALESCE(VALUES(`user.need_recommend`),`muyangquandebaba_post_comments`.`user.need_recommend`),
`muyangquandebaba_post_comments`.`user.nickname` = COALESCE(VALUES(`user.nickname`),`muyangquandebaba_post_comments`.`user.nickname`),
`muyangquandebaba_post_comments`.`user.platform_sync_info` = COALESCE(VALUES(`user.platform_sync_info`),`muyangquandebaba_post_comments`.`user.platform_sync_info`),
`muyangquandebaba_post_comments`.`user.prevent_download` = COALESCE(VALUES(`user.prevent_download`),`muyangquandebaba_post_comments`.`user.prevent_download`),
`muyangquandebaba_post_comments`.`user.react_setting` = COALESCE(VALUES(`user.react_setting`),`muyangquandebaba_post_comments`.`user.react_setting`),
`muyangquandebaba_post_comments`.`user.reflow_page_gid` = COALESCE(VALUES(`user.reflow_page_gid`),`muyangquandebaba_post_comments`.`user.reflow_page_gid`),
`muyangquandebaba_post_comments`.`user.reflow_page_uid` = COALESCE(VALUES(`user.reflow_page_uid`),`muyangquandebaba_post_comments`.`user.reflow_page_uid`),
`muyangquandebaba_post_comments`.`user.region` = COALESCE(VALUES(`user.region`),`muyangquandebaba_post_comments`.`user.region`),
`muyangquandebaba_post_comments`.`user.relative_users` = COALESCE(VALUES(`user.relative_users`),`muyangquandebaba_post_comments`.`user.relative_users`),
`muyangquandebaba_post_comments`.`user.search_highlight` = COALESCE(VALUES(`user.search_highlight`),`muyangquandebaba_post_comments`.`user.search_highlight`),
`muyangquandebaba_post_comments`.`user.sec_uid` = COALESCE(VALUES(`user.sec_uid`),`muyangquandebaba_post_comments`.`user.sec_uid`),
`muyangquandebaba_post_comments`.`user.secret` = COALESCE(VALUES(`user.secret`),`muyangquandebaba_post_comments`.`user.secret`),
`muyangquandebaba_post_comments`.`user.shield_comment_notice` = COALESCE(VALUES(`user.shield_comment_notice`),`muyangquandebaba_post_comments`.`user.shield_comment_notice`),
`muyangquandebaba_post_comments`.`user.shield_digg_notice` = COALESCE(VALUES(`user.shield_digg_notice`),`muyangquandebaba_post_comments`.`user.shield_digg_notice`),
`muyangquandebaba_post_comments`.`user.shield_edit_field_info` = COALESCE(VALUES(`user.shield_edit_field_info`),`muyangquandebaba_post_comments`.`user.shield_edit_field_info`),
`muyangquandebaba_post_comments`.`user.shield_follow_notice` = COALESCE(VALUES(`user.shield_follow_notice`),`muyangquandebaba_post_comments`.`user.shield_follow_notice`),
`muyangquandebaba_post_comments`.`user.short_id` = COALESCE(VALUES(`user.short_id`),`muyangquandebaba_post_comments`.`user.short_id`),
`muyangquandebaba_post_comments`.`user.show_image_bubble` = COALESCE(VALUES(`user.show_image_bubble`),`muyangquandebaba_post_comments`.`user.show_image_bubble`),
`muyangquandebaba_post_comments`.`user.signature` = COALESCE(VALUES(`user.signature`),`muyangquandebaba_post_comments`.`user.signature`),
`muyangquandebaba_post_comments`.`user.special_account.special_account_list` = COALESCE(VALUES(`user.special_account.special_account_list`),`muyangquandebaba_post_comments`.`user.special_account.special_account_list`),
`muyangquandebaba_post_comments`.`user.special_lock` = COALESCE(VALUES(`user.special_lock`),`muyangquandebaba_post_comments`.`user.special_lock`),
`muyangquandebaba_post_comments`.`user.status` = COALESCE(VALUES(`user.status`),`muyangquandebaba_post_comments`.`user.status`),
`muyangquandebaba_post_comments`.`user.stitch_setting` = COALESCE(VALUES(`user.stitch_setting`),`muyangquandebaba_post_comments`.`user.stitch_setting`),
`muyangquandebaba_post_comments`.`user.total_favorited` = COALESCE(VALUES(`user.total_favorited`),`muyangquandebaba_post_comments`.`user.total_favorited`),
`muyangquandebaba_post_comments`.`user.twitter_id` = COALESCE(VALUES(`user.twitter_id`),`muyangquandebaba_post_comments`.`user.twitter_id`),
`muyangquandebaba_post_comments`.`user.twitter_name` = COALESCE(VALUES(`user.twitter_name`),`muyangquandebaba_post_comments`.`user.twitter_name`),
`muyangquandebaba_post_comments`.`user.type_label` = COALESCE(VALUES(`user.type_label`),`muyangquandebaba_post_comments`.`user.type_label`),
`muyangquandebaba_post_comments`.`user.uid` = COALESCE(VALUES(`user.uid`),`muyangquandebaba_post_comments`.`user.uid`),
`muyangquandebaba_post_comments`.`user.unique_id` = COALESCE(VALUES(`user.unique_id`),`muyangquandebaba_post_comments`.`user.unique_id`),
`muyangquandebaba_post_comments`.`user.unique_id_modify_time` = COALESCE(VALUES(`user.unique_id_modify_time`),`muyangquandebaba_post_comments`.`user.unique_id_modify_time`),
`muyangquandebaba_post_comments`.`user.user_canceled` = COALESCE(VALUES(`user.user_canceled`),`muyangquandebaba_post_comments`.`user.user_canceled`),
`muyangquandebaba_post_comments`.`user.user_mode` = COALESCE(VALUES(`user.user_mode`),`muyangquandebaba_post_comments`.`user.user_mode`),
`muyangquandebaba_post_comments`.`user.user_period` = COALESCE(VALUES(`user.user_period`),`muyangquandebaba_post_comments`.`user.user_period`),
`muyangquandebaba_post_comments`.`user.user_profile_guide` = COALESCE(VALUES(`user.user_profile_guide`),`muyangquandebaba_post_comments`.`user.user_profile_guide`),
`muyangquandebaba_post_comments`.`user.user_rate` = COALESCE(VALUES(`user.user_rate`),`muyangquandebaba_post_comments`.`user.user_rate`),
`muyangquandebaba_post_comments`.`user.user_tags` = COALESCE(VALUES(`user.user_tags`),`muyangquandebaba_post_comments`.`user.user_tags`),
`muyangquandebaba_post_comments`.`user.verification_type` = COALESCE(VALUES(`user.verification_type`),`muyangquandebaba_post_comments`.`user.verification_type`),
`muyangquandebaba_post_comments`.`user.verify_info` = COALESCE(VALUES(`user.verify_info`),`muyangquandebaba_post_comments`.`user.verify_info`),
`muyangquandebaba_post_comments`.`user.video_icon.height` = COALESCE(VALUES(`user.video_icon.height`),`muyangquandebaba_post_comments`.`user.video_icon.height`),
`muyangquandebaba_post_comments`.`user.video_icon.uri` = COALESCE(VALUES(`user.video_icon.uri`),`muyangquandebaba_post_comments`.`user.video_icon.uri`),
`muyangquandebaba_post_comments`.`user.video_icon.url_list` = COALESCE(VALUES(`user.video_icon.url_list`),`muyangquandebaba_post_comments`.`user.video_icon.url_list`),
`muyangquandebaba_post_comments`.`user.video_icon.url_prefix` = COALESCE(VALUES(`user.video_icon.url_prefix`),`muyangquandebaba_post_comments`.`user.video_icon.url_prefix`),
`muyangquandebaba_post_comments`.`user.video_icon.width` = COALESCE(VALUES(`user.video_icon.width`),`muyangquandebaba_post_comments`.`user.video_icon.width`),
`muyangquandebaba_post_comments`.`user.white_cover_url` = COALESCE(VALUES(`user.white_cover_url`),`muyangquandebaba_post_comments`.`user.white_cover_url`),
`muyangquandebaba_post_comments`.`user.with_commerce_entry` = COALESCE(VALUES(`user.with_commerce_entry`),`muyangquandebaba_post_comments`.`user.with_commerce_entry`),
`muyangquandebaba_post_comments`.`user.with_fusion_shop_entry` = COALESCE(VALUES(`user.with_fusion_shop_entry`),`muyangquandebaba_post_comments`.`user.with_fusion_shop_entry`),
`muyangquandebaba_post_comments`.`user.with_shop_entry` = COALESCE(VALUES(`user.with_shop_entry`),`muyangquandebaba_post_comments`.`user.with_shop_entry`),
`muyangquandebaba_post_comments`.`user.youtube_channel_id` = COALESCE(VALUES(`user.youtube_channel_id`),`muyangquandebaba_post_comments`.`user.youtube_channel_id`),
`muyangquandebaba_post_comments`.`user.youtube_channel_title` = COALESCE(VALUES(`user.youtube_channel_title`),`muyangquandebaba_post_comments`.`user.youtube_channel_title`),
`muyangquandebaba_post_comments`.`user.youtube_expire_time` = COALESCE(VALUES(`user.youtube_expire_time`),`muyangquandebaba_post_comments`.`user.youtube_expire_time`),
`muyangquandebaba_post_comments`.`user_buried` = COALESCE(VALUES(`user_buried`),`muyangquandebaba_post_comments`.`user_buried`),
`muyangquandebaba_post_comments`.`user_digged` = COALESCE(VALUES(`user_digged`),`muyangquandebaba_post_comments`.`user_digged`);
         """)


        drop_post_comments_temp_table = text("""
        Drop Table muyangquandebaba_post_comments_temp
        """)


        try:
            with engine.connect() as connection:
             connection.execute(sql_insert_into_another_table)
             connection.execute(drop_post_comments_temp_table)

            print(f'Successfully stored comments for aweme_id: {aweme_id}')

        except Exception as e:
            print(f'fail to store comments for aweme_id: {aweme_id}')
            with engine.connect() as connection:
               connection.execute(drop_post_comments_temp_table)
        continue
    else:
        continue